<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/test-hosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mt5

!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [10]:
# install package
# mount to drive if dataset in google drive

from google.colab import drive
drive.mount('/content/gdrive')

!pip install simpletransformers
!pip install torch
!pip install pythainlp
!pip install python-crfsuite

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 16.1 MB/s eta 0:00:00


In [5]:
# load model

import logging
import torch
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 100
model_args.length_penalty = 1
model_args.num_beams = 10

device = torch.cuda.is_available()

# path model
PATH = '/content/gdrive/MyDrive/model/mt5-4bs-4e-5lr'

model = T5Model("mt5", PATH, args=model_args, use_cuda=device)

In [11]:
import re
import pycrfsuite
from pythainlp import sent_tokenize

TH_predict = 'โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึกโดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector) ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์ ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano'
EN_predict = 'This engineering project develops a system for detecting. And classifying plant disease from photographs of melon leaves using deep learning. The system uses NVIDIA Jetson Nano to process images from cameras in the greenhouse. To develop a deep learning model, the images of the melon leaves from the greenhouses were collected. And the melon leaf disease were labelled. Then use the dataset to train the object detector model. Which will be able to detect disease in melon leaves. and when the system detects a disease, the caretaker will be notified of the issue through the LINE application. In the process of selecting a deep learning model to be deployed on the Jetson Nano.'

# Thai cut sentence
TH_predict = sent_tokenize(TH_predict)
for i in range(len(TH_predict)):
  # remove space in back
  TH_predict[i] = TH_predict[i].rstrip()

  # debug for engine fail example ['750ml', '1500ml']
  # move to append previous sentences
  # and replace ''
  if re.search('[0-9]', TH_predict[i][0]):
    j = i
    while True:
      if TH_predict[j-1] != '':
        TH_predict[j-1] += ' ' + TH_predict[i]
        break
    j -= 1
    TH_predict[i] = ''

# remove '' in list
while '' in TH_predict:
    TH_predict.remove('')

# English cut sentence
EN_predict = EN_predict.split('. ')
for i in range(len(EN_predict)):
  if EN_predict[i][-1] != '.':
    EN_predict[i] += '.'

In [12]:
TH_predict

['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึกโดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
 'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector) ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
 'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์ ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']

In [13]:
EN_predict

['This engineering project develops a system for detecting.',
 'And classifying plant disease from photographs of melon leaves using deep learning.',
 'The system uses NVIDIA Jetson Nano to process images from cameras in the greenhouse.',
 'To develop a deep learning model, the images of the melon leaves from the greenhouses were collected.',
 'And the melon leaf disease were labelled.',
 'Then use the dataset to train the object detector model.',
 'Which will be able to detect disease in melon leaves.',
 'and when the system detects a disease, the caretaker will be notified of the issue through the LINE application.',
 'In the process of selecting a deep learning model to be deployed on the Jetson Nano.']

In [14]:
# model prediction
TH_predictions = model.predict(TH_predict)
print('--------------------------------------------------------------')
print('Translation TH to EN')
for i, info in enumerate(TH_predictions):
  print(TH_predict[i] + '\n' + info + '\n')
print('--------------------------------------------------------------')
EN_predictions = model.predict(EN_predict)
print('Translation EN to TH')
for i, info in enumerate(EN_predictions):
  print(EN_predict[i] + '\n' + info + '\n')
print('--------------------------------------------------------------')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3732: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/3 [00:00<?, ?it/s]

--------------------------------------------------------------
Translation TH to EN
โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึกโดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน
This engineering project is to develop a classification system and classification of plant diseases from melon leaves images with deep learning using the Jessen Nano machine to processing image from a camera in a house.

ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector) ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน
The in the development of the deep learning model, the images of melon leaves were collected from the house and created the signs of the melon faults, then the data set was used to simulate the object detection model (Object Detection) that can detect the defects of melon leaves.

และเมื่อ

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/9 [00:00<?, ?it/s]

Translation EN to TH
This engineering project develops a system for detecting.
โครงการวิศวกรรมนี้พัฒนาระบบตรวจจับ

And classifying plant disease from photographs of melon leaves using deep learning.
และจําแนกโรคพืชจากภาพถ่ายใบมะกรูดโดยใช้การเรียนรู้ลึก

The system uses NVIDIA Jetson Nano to process images from cameras in the greenhouse.
ระบบใช้ NVIDIAjetsonNANO เพื่อประมวลผลภาพจากกล้องในโรงเรือน

To develop a deep learning model, the images of the melon leaves from the greenhouses were collected.
เพื่อพัฒนาแบบจําลองการเรียนรู้ลึก รูปภาพใบมะกรูดจากโรงเรือนปลูกถูกรวบรวม

And the melon leaf disease were labelled.
และฉลากโรคใบมะกรูด

Then use the dataset to train the object detector model.
จากนั้นใช้ข้อมูลเพื่อ train โมเดลตัวตรวจจับวัตถุ

Which will be able to detect disease in melon leaves.
ซึ่งจะสามารถตรวจจับโรคในใบมะกรูดได้

and when the system detects a disease, the caretaker will be notified of the issue through the LINE application.
และเมื่อระบบตรวจพบโรค ผู้ดูแลจะแจ้งปัญหาผ่านแอปพลิเ